In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Lý thuyết AI&DS - Tự học/Machine Learning/Đồ án cơ sở/Code ĐACS - Giai đoạn 3")

In [ ]:
import spacy
import nltk
import requests

In [ ]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')      # chứa: userId, movieId, rating, timestamp
movies = pd.read_csv('movies.csv')        # chứa: movieId, title, genres
tags = pd.read_csv('tags.csv')            # chứa: userId, movieId, tag, timestamp
links = pd.read_csv('links.csv')          # chứa: movieId, imdbId, tmdbId

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x: x.split('|') if isinstance(x, str) else [])
ratings_movies = pd.merge(ratings, movies, on='movieId', how='left')

In [ ]:
import pandas as pd

# Chuyển đổi cột timestamp sang datetime (theo đơn vị giây)
tags['datetime'] = pd.to_datetime(tags['timestamp'], unit='s')


print("Kích thước bảng tags:", tags.shape)
tags.head(3683)

Kích thước bảng tags: (3683, 5)


,userId,movieId,tag,timestamp,datetime
0,2,60756,funny,1445714994,2015-10-24 19:29:54
1,2,60756,Highly quotable,1445714996,2015-10-24 19:29:56
2,2,60756,will ferrell,1445714992,2015-10-24 19:29:52
3,2,89774,Boxing story,1445715207,2015-10-24 19:33:27
4,2,89774,MMA,1445715200,2015-10-24 19:33:20
...,...,...,...,...,...
3678,606,7382,for katie,1171234019,2007-02-11 22:46:59
3679,606,7936,austere,1173392334,2007-03-08 22:18:54
3680,610,3265,gun fu,1493843984,2017-05-03 20:39:44
3681,610,3265,heroic bloodshed,1493843978,2017-05-03 20:39:38


In [ ]:
# Chuyển tag về chữ thường
tags['tag'] = tags['tag'].str.lower()

# Xóa khoảng trắng đầu/cuối (nếu có)
tags['tag'] = tags['tag'].str.strip()

tags.head(3683)

,userId,movieId,tag,timestamp,datetime
0,2,60756,funny,1445714994,2015-10-24 19:29:54
1,2,60756,highly quotable,1445714996,2015-10-24 19:29:56
2,2,60756,will ferrell,1445714992,2015-10-24 19:29:52
3,2,89774,boxing story,1445715207,2015-10-24 19:33:27
4,2,89774,mma,1445715200,2015-10-24 19:33:20
...,...,...,...,...,...
3678,606,7382,for katie,1171234019,2007-02-11 22:46:59
3679,606,7936,austere,1173392334,2007-03-08 22:18:54
3680,610,3265,gun fu,1493843984,2017-05-03 20:39:44
3681,610,3265,heroic bloodshed,1493843978,2017-05-03 20:39:38


In [ ]:
nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    """Nhận diện thực thể có tên trong tag."""
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

def extract_nouns(text):
    """Trích xuất danh từ từ tag."""
    doc = nlp(text)
    return [token.text for token in doc if token.pos_ == 'NOUN']

# Áp dụng cho cột tag
tags['entities'] = tags['tag'].apply(lambda x: extract_entities(x))
tags['nouns'] = tags['tag'].apply(lambda x: extract_nouns(x))

# Phân tích cảm xúc với VADER từ NLTK
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    """Lấy chỉ số cảm xúc của tag."""
    return sia.polarity_scores(text)

tags['sentiment'] = tags['tag'].apply(lambda x: get_sentiment(x))

# Entity Linking với Wikidata
def link_entity_wikidata(tag):
    """
    Dùng Wikidata API để tìm kiếm thực thể phù hợp với tag.
    Trả về dict chứa id, label và description của thực thể nếu tìm được.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "search": tag,
        "language": "en",
        "format": "json"
    }
    try:
        response = requests.get(url, params=params, timeout=5)
        if response.status_code == 200:
            data = response.json()
            if "search" in data and data["search"]:
                # Trả về kết quả đầu tiên (có thể cải tiến bằng cách xem thêm điểm số)
                result = data["search"][0]
                return {
                    "id": result.get("id"),
                    "label": result.get("label"),
                    "description": result.get("description")
                }
    except Exception as e:
        print(f"Lỗi khi tìm kiếm entity cho tag '{tag}':", e)
    return {}

tags["wikidata"] = tags["tag"].apply(link_entity_wikidata)

# Hiển thị kết quả
print("Kích thước bảng tags:", tags.shape)
tags.head(3683)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Kích thước bảng tags: (3683, 9)


,userId,movieId,tag,timestamp,datetime,entities,nouns,sentiment,wikidata
0,2,60756,funny,1445714994,2015-10-24 19:29:54,[],[],"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...","{'id': 'Q1760610', 'label': 'comic book', 'des..."
1,2,60756,highly quotable,1445714996,2015-10-24 19:29:56,[],[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",{}
2,2,60756,will ferrell,1445714992,2015-10-24 19:29:52,[],[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'id': 'Q218503', 'label': 'Will Ferrell', 'de..."
3,2,89774,boxing story,1445715207,2015-10-24 19:33:27,[],"[boxing, story]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",{}
4,2,89774,mma,1445715200,2015-10-24 19:33:20,[],[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'id': 'Q160236', 'label': 'Metropolitan Museu..."
...,...,...,...,...,...,...,...,...,...
3678,606,7382,for katie,1171234019,2007-02-11 22:46:59,[],[katie],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'id': 'Q81561209', 'label': 'For Katie', 'des..."
3679,606,7936,austere,1173392334,2007-03-08 22:18:54,[],[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'id': 'Q3629919', 'label': 'Austere', 'descri..."
3680,610,3265,gun fu,1493843984,2017-05-03 20:39:44,[],"[gun, fu]","{'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound...","{'id': 'Q2637123', 'label': 'Gun Kata', 'descr..."
3681,610,3265,heroic bloodshed,1493843978,2017-05-03 20:39:38,[],[],"{'neg': 0.0, 'neu': 0.217, 'pos': 0.783, 'comp...","{'id': 'Q622310', 'label': 'heroic bloodshed',..."


from matplotlib import pyplot as plt
_df_0['userId'].plot(kind='hist', bins=20, title='userId')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['movieId'].plot(kind='hist', bins=20, title='movieId')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['userId']
  ys = series['movieId']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_7.sort_values('userId', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('userId')
_ = plt.ylabel('movieId')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['userId']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'userId'}, axis=1)
              .sort_values('userId', ascending=True))
  xs = counted['userId']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_8.sort_values('userId', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('userId')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_9['userId'].plot(kind='line', figsize=(8, 4), title='userId')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['movieId'].plot(kind='line', figsize=(8, 4), title='movieId')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['timestamp'].plot(kind='line', figsize=(8, 4), title='timestamp')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
tags.to_pickle('tags_preprocessed.pkl')
print("Đã lưu DataFrame vào file 'tags_preprocessed.pkl'.")

Đã lưu DataFrame vào file 'tags_preprocessed.pkl'.


In [ ]:
def infer_entity_label_from_wikidata(wikidata):
    """
    Dựa vào mô tả của wikidata để suy ra nhãn phù hợp cho thực thể.
    Trả về "PERSON" nếu mô tả chứa các từ khóa liên quan đến người,
    "WORK_OF_ART" nếu mô tả liên quan đến tác phẩm (như comic, film, book),
    hoặc "CONCEPT"/"ENTITY" cho các trường hợp khác.
    """
    if not wikidata:
        return "ENTITY"
    description = wikidata.get("description", "")
    description = description.lower() if description else ""
    if any(word in description for word in ["actor", "actress", "person", "director", "celebrity"]):
        return "PERSON"
    elif any(word in description for word in ["film", "movie", "comic", "book", "novel", "artwork"]):
        return "WORK_OF_ART"
    elif "drug" in description:
        return "CONCEPT"
    else:
        return "ENTITY"

def merge_entities(entities, wikidata):
    """
    Kết hợp kết quả thực thể từ spaCy và Wikidata.
    Nếu Wikidata trả về thông tin và label của nó chưa xuất hiện trong entities,
    thêm vào danh sách với nhãn được suy ra từ mô tả của Wikidata.
    """
    merged = entities.copy()  # Bắt đầu với các thực thể từ spaCy
    if wikidata and wikidata.get("id"):
        wikidata_label = wikidata.get("label", "")
        if not any(wikidata_label.lower() in ent[0].lower() for ent in merged):
            inferred_label = infer_entity_label_from_wikidata(wikidata)
            merged.append((wikidata_label, inferred_label))
    return merged

# Áp dụng hàm merge_entities cho từng dòng của DataFrame
tags['merged_entities'] = tags.apply(lambda row: merge_entities(row['entities'], row['wikidata']), axis=1)

# Hiển thị kết quả
print(tags[['tag', 'entities', 'wikidata', 'merged_entities']].head(10))


                 tag                       entities  \
0              funny                             []   
1    highly quotable                             []   
2       will ferrell                             []   
3       boxing story                             []   
4                mma                             []   
5          tom hardy                             []   
6              drugs                             []   
7  leonardo dicaprio  [(leonardo dicaprio, PERSON)]   
8    martin scorsese    [(martin scorsese, PERSON)]   
9       way too long                             []   

                                            wikidata  \
0  {'id': 'Q1760610', 'label': 'comic book', 'des...   
1                                                 {}   
2  {'id': 'Q218503', 'label': 'Will Ferrell', 'de...   
3                                                 {}   
4  {'id': 'Q160236', 'label': 'Metropolitan Museu...   
5  {'id': 'Q208026', 'label': 'Tom Hardy', 'descr...   
6 